In [ ]:
!pip install transformers datasets torch scikit-learn pandas numpy matplotlib seaborn sentence-transformers

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import json
import random

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

def generate_simple_tickets(n_samples=200):
    categories = ['Technical', 'Billing', 'Account', 'Feature Request', 'Bug Report', 'Login']

    templates = {
        'Technical': ["System not working", "Technical error occurred", "Can't access features"],
        'Billing': ["Wrong charge amount", "Billing question", "Payment failed"],
        'Account': ["Update profile", "Change settings", "Account locked"],
        'Feature Request': ["Add new feature", "Improve functionality", "Need better tools"],
        'Bug Report': ["Found a bug", "Error in system", "Something broken"],
        'Login': ["Can't login", "Password reset", "Login failed"]
    }

    tickets = []
    for _ in range(n_samples):
        category = random.choice(categories)
        base_text = random.choice(templates[category])
        variations = [
            f"{base_text} in the application",
            f"Help with {base_text.lower()}",
            f"Issue: {base_text.lower()}",
            f"Problem with {base_text.lower()}",
            base_text
        ]
        ticket_text = random.choice(variations)
        tickets.append({'text': ticket_text, 'category': category})

    return pd.DataFrame(tickets)

print("Generating small dataset...")
df = generate_simple_tickets(200)
print("Dataset created!")
print("\nCategory Distribution:")
print(df['category'].value_counts())

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])
num_labels = len(label_encoder.classes_)

X_train, X_temp, y_train, y_temp = train_test_split(
    df['text'], df['label'], test_size=0.4, random_state=42, stratify=df['label']
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

train_df = pd.DataFrame({'text': X_train, 'label': y_train})
val_df = pd.DataFrame({'text': X_val, 'label': y_val})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

class FastZeroShotClassifier:
    def __init__(self):
        print("Loading lightweight sentence transformer...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.categories = label_encoder.classes_.tolist()
        self.category_embeddings = self.model.encode(self.categories)

    def predict(self, texts, top_k=3):
        if isinstance(texts, str):
            texts = [texts]

        text_embeddings = self.model.encode(texts)
        similarities = cosine_similarity(text_embeddings, self.category_embeddings)

        predictions = []
        for sim_scores in similarities:
            top_indices = np.argsort(sim_scores)[::-1][:top_k]
            top_predictions = [(self.categories[idx], sim_scores[idx]) for idx in top_indices]
            predictions.append(top_predictions)

        return predictions

    def predict_single_label(self, texts):
        predictions = self.predict(texts, top_k=1)
        single_preds = []
        for pred in predictions:
            label_name = pred[0][0]
            label_idx = label_encoder.transform([label_name])[0]
            single_preds.append(label_idx)
        return single_preds

class FastFewShotClassifier:
    def __init__(self):
        print("Loading few-shot classifier...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.categories = label_encoder.classes_.tolist()
        self.examples = self._create_examples()

    def _create_examples(self):
        examples = {}
        for category in self.categories:
            category_samples = train_df[train_df['label'] == label_encoder.transform([category])[0]]
            if len(category_samples) > 0:
                examples[category] = category_samples['text'].iloc[0]
            else:
                examples[category] = f"Sample {category.lower()} ticket"
        return examples

    def predict(self, texts, top_k=3):
        if isinstance(texts, str):
            texts = [texts]

        enhanced_categories = []
        for cat in self.categories:
            enhanced_cat = f"{cat}: {self.examples[cat]}"
            enhanced_categories.append(enhanced_cat)

        text_embeddings = self.model.encode(texts)
        category_embeddings = self.model.encode(enhanced_categories)
        similarities = cosine_similarity(text_embeddings, category_embeddings)

        predictions = []
        for sim_scores in similarities:
            top_indices = np.argsort(sim_scores)[::-1][:top_k]
            top_predictions = [(self.categories[idx], sim_scores[idx]) for idx in top_indices]
            predictions.append(top_predictions)

        return predictions

    def predict_single_label(self, texts):
        predictions = self.predict(texts, top_k=1)
        single_preds = []
        for pred in predictions:
            label_name = pred[0][0]
            label_idx = label_encoder.transform([label_name])[0]
            single_preds.append(label_idx)
        return single_preds

class FastFineTunedClassifier:
    def __init__(self, model_name="distilbert-base-uncased"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = None

    def preprocess_data(self, texts, labels=None):
        encodings = self.tokenizer(
            texts.tolist() if hasattr(texts, 'tolist') else list(texts),
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )

        if labels is not None:
            encodings['labels'] = torch.tensor(labels.tolist() if hasattr(labels, 'tolist') else list(labels))

        return Dataset.from_dict(encodings)

    def train(self, train_texts, train_labels, val_texts, val_labels):
        print("Loading model for fine-tuning...")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=num_labels
        )

        train_dataset = self.preprocess_data(train_texts, train_labels)
        val_dataset = self.preprocess_data(val_texts, val_labels)

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=1,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=32,
            warmup_steps=50,
            weight_decay=0.01,
            logging_dir='./logs',
            evaluation_strategy="no",
            save_strategy="no",
            logging_steps=20,
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
        )

        print("Training model...")
        trainer.train()
        self.model = trainer.model

    def predict(self, texts, top_k=3):
        if self.model is None:
            raise ValueError("Model not trained yet")

        inputs = self.tokenizer(
            texts.tolist() if hasattr(texts, 'tolist') else list(texts),
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

        results = []
        for pred in predictions:
            top_indices = torch.topk(pred, top_k).indices
            top_scores = torch.topk(pred, top_k).values

            top_predictions = []
            for idx, score in zip(top_indices, top_scores):
                label = label_encoder.inverse_transform([idx.item()])[0]
                top_predictions.append((label, score.item()))

            results.append(top_predictions)

        return results

    def predict_single_label(self, texts):
        if self.model is None:
            raise ValueError("Model not trained yet")

        inputs = self.tokenizer(
            texts.tolist() if hasattr(texts, 'tolist') else list(texts),
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        return predictions.tolist()

def evaluate_classifier(classifier, test_texts, test_labels, classifier_name):
    print(f"\n{classifier_name} Evaluation:")
    print("="*40)

    y_pred = classifier.predict_single_label(test_texts)

    accuracy = accuracy_score(test_labels, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    return accuracy, y_pred

def evaluate_top_k_predictions(classifier, test_texts, test_labels, classifier_name, k=3):
    print(f"\n{classifier_name} Top-{k} Predictions:")

    predictions = classifier.predict(test_texts, top_k=k)

    for i in range(1, k+1):
        correct = 0
        for pred_list, true_label in zip(predictions, test_labels):
            true_label_name = label_encoder.inverse_transform([true_label])[0]
            top_i_labels = [pred[0] for pred in pred_list[:i]]
            if true_label_name in top_i_labels:
                correct += 1

        accuracy = correct / len(test_labels)
        print(f"Top-{i} Accuracy: {accuracy:.4f}")

print("\n" + "="*50)
print("FAST EVALUATION STARTING")
print("="*50)

print("\nInitializing Zero-Shot Classifier...")
zero_shot = FastZeroShotClassifier()

print("Evaluating Zero-Shot...")
zero_shot_acc, zero_shot_pred = evaluate_classifier(zero_shot, X_test, y_test, "Zero-Shot")
evaluate_top_k_predictions(zero_shot, X_test, y_test, "Zero-Shot", k=3)

print("\nInitializing Few-Shot Classifier...")
few_shot = FastFewShotClassifier()

print("Evaluating Few-Shot...")
few_shot_acc, few_shot_pred = evaluate_classifier(few_shot, X_test, y_test, "Few-Shot")
evaluate_top_k_predictions(few_shot, X_test, y_test, "Few-Shot", k=3)

print("\nTraining Fine-Tuned Classifier...")
fine_tuned = FastFineTunedClassifier()
fine_tuned.train(X_train, y_train, X_val, y_val)

print("Evaluating Fine-Tuned...")
fine_tuned_acc, fine_tuned_pred = evaluate_classifier(fine_tuned, X_test, y_test, "Fine-Tuned")
evaluate_top_k_predictions(fine_tuned, X_test, y_test, "Fine-Tuned", k=3)

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
accuracies = [zero_shot_acc, few_shot_acc, fine_tuned_acc]
methods = ['Zero-Shot', 'Few-Shot', 'Fine-Tuned']
bars = plt.bar(methods, accuracies, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Classification Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{acc:.3f}', ha='center', va='bottom')

plt.subplot(1, 2, 2)
categories = label_encoder.classes_
category_counts = [sum(y_test == i) for i in range(len(categories))]
plt.pie(category_counts, labels=categories, autopct='%1.1f%%')
plt.title('Test Set Distribution')

plt.tight_layout()
plt.show()

def demonstrate_predictions(sample_texts, classifiers, classifier_names):
    print("\nSample Predictions:")
    print("="*60)

    for i, text in enumerate(sample_texts):
        print(f"\nTicket {i+1}: '{text}'")
        print("-" * 40)

        for classifier, name in zip(classifiers, classifier_names):
            predictions = classifier.predict([text], top_k=3)[0]
            print(f"{name:12}: ", end="")
            for j, (label, score) in enumerate(predictions):
                print(f"{j+1}.{label}({score:.2f})", end="  ")
            print()

sample_tickets = [
    "Can't login to my account",
    "System is very slow",
    "Wrong billing amount charged",
    "Need new export feature",
    "Found bug in dashboard"
]

demonstrate_predictions(
    sample_tickets,
    [zero_shot, few_shot, fine_tuned],
    ["Zero-Shot", "Few-Shot", "Fine-Tuned"]
)

results_summary = {
    'Method': ['Zero-Shot', 'Few-Shot', 'Fine-Tuned'],
    'Accuracy': [zero_shot_acc, few_shot_acc, fine_tuned_acc]
}

results_df = pd.DataFrame(results_summary)
print("\nFinal Results:")
print("="*30)
print(results_df.to_string(index=False))

print(f"\nBest performing method: {results_df.loc[results_df['Accuracy'].idxmax(), 'Method']}")
print("All evaluations completed successfully!")